## Working with Transformers in the HuggingFace Ecosystem

In this laboratory exercise we will learn how to work with the HuggingFace ecosystem to adapt models to new tasks. As you will see, much of what is required is *investigation* into the inner-workings of the HuggingFace abstractions. With a little work, a little trial-and-error, it is fairly easy to get a working adaptation pipeline up and running.

### Exercise 1: Sentiment Analysis (warm up)

In this first exercise we will start from a pre-trained BERT transformer and build up a model able to perform text sentiment analysis. Transformers are complex beasts, so we will build up our pipeline in several explorative and incremental steps.

#### Exercise 1.1: Dataset Splits and Pre-trained model
There are a many sentiment analysis datasets, but we will use one of the smallest ones available: the [Cornell Rotten Tomatoes movie review dataset](cornell-movie-review-data/rotten_tomatoes), which consists of 5,331 positive and 5,331 negative processed sentences from the Rotten Tomatoes movie reviews.

**Your first task**: Load the dataset and figure out what splits are available and how to get them. Spend some time exploring the dataset to see how it is organized. Note that we will be using the [HuggingFace Datasets](https://huggingface.co/docs/datasets/en/index) library for downloading, accessing, splitting, and batching data for training and evaluation.

In [ ]:
# Import necessary libraries

import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
import time
import copy
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from datasets import load_dataset, get_dataset_split_names
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from peft import LoraConfig, TaskType, get_peft_model

import numpy as np
import math

from tqdm import tqdm
from sklearn import svm
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

/home/marco/miniforge3/envs/DLA/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Set the random seed for reproducibility

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True

In [ ]:
# Path configurations

data_dir = './data' 
checkpoints_dir = './checkpoints'
log_dir = './logs'

batch = 64
n_examples_tokenizer = 5
n_examples_test = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Clear directories for data, checkpoints, and logs

import shutil
import os

def clear(_dir):
    if os.path.exists(_dir):
        if os.path.isdir(_dir):
            shutil.rmtree(_dir)
        os.makedirs(_dir, exist_ok = True)

# Uncomment the following lines to clear the directories

#clear(data_dir)
#clear(checkpoints_dir)
#clear(log_dir)

In [ ]:
# TensorBoard setup

%load_ext tensorboard
%tensorboard --logdir=./logs

In [ ]:
# Rotten Tomatoes dataset loading
# Using the Cornell Movie Review Data for Rotten Tomatoes sentiment analysis
# This dataset is available in the Hugging Face datasets library

dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")

print(dataset.shape)
print(dataset["train"][0])

data_loaders = []

for split in dataset.keys():
    data_loaders.append(DataLoader(dataset[split], batch_size = batch, shuffle = False))

train, val, test = data_loaders

{'train': (8530, 2), 'validation': (1066, 2), 'test': (1066, 2)}
{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}


#### Exercise 1.2: A Pre-trained BERT and Tokenizer

The model we will use is a *very* small BERT transformer called [Distilbert](https://huggingface.co/distilbert/distilbert-base-uncased) this model was trained (using self-supervised learning) on the same corpus as BERT but using the full BERT base model as a *teacher*.

**Your next task**: Load the Distilbert model and corresponding tokenizer. Use the tokenizer on a few samples from the dataset and pass the tokens through the model to see what outputs are provided. I suggest you use the [`AutoModel`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) class (and the `from_pretrained()` method) to load the model and `AutoTokenizer` to load the tokenizer).

In [ ]:
# Test the tokenizer with a few examples

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

for i in range(n_examples_tokenizer):
    example = dataset["train"][i]["text"]

    ttt = tokenizer(example, return_tensors = "pt", truncation = True, padding = True)
    tokens = tokenizer.convert_ids_to_tokens(ttt["input_ids"][0])
    print(tokens)

print()

test_text = "Pippo was so silly at the beginning of the movie, he should had die since min. 1" #custom text to test

ttt = tokenizer(test_text, return_tensors = "pt", truncation = True, padding = True) #test tokenized text
#pt = pytorch (tensors)


print(ttt.keys())

tokens_ids = tokenizer.convert_ids_to_tokens(ttt["input_ids"][0])
print(tokens_ids)

['[CLS]', 'the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', "'", 's', 'new', '"', 'conan', '"', 'and', 'that', 'he', "'", 's', 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarz', '##ene', '##gger', ',', 'jean', '-', 'cl', '##aud', 'van', 'dam', '##me', 'or', 'steven', 'sega', '##l', '.', '[SEP]']
['[CLS]', 'the', 'gorgeous', '##ly', 'elaborate', 'continuation', 'of', '"', 'the', 'lord', 'of', 'the', 'rings', '"', 'trilogy', 'is', 'so', 'huge', 'that', 'a', 'column', 'of', 'words', 'cannot', 'adequately', 'describe', 'co', '-', 'writer', '/', 'director', 'peter', 'jackson', "'", 's', 'expanded', 'vision', 'of', 'j', '.', 'r', '.', 'r', '.', 'tolkien', "'", 's', 'middle', '-', 'earth', '.', '[SEP]']
['[CLS]', 'effective', 'but', 'too', '-', 'te', '##pid', 'bio', '##pic', '[SEP]']
['[CLS]', 'if', 'you', 'sometimes', 'like', 'to', 'go', 'to', 'the', 'movies', 'to', 'have', 'fun', ',', 'was', '##abi', 'is', 'a', 'good', 'place', 'to', '

In [ ]:
# Load the pre-trained DistilBERT model for sequence classification and pass the tokens

model = AutoModel.from_pretrained("distilbert-base-uncased", num_labels = 2) #0 for bad reviews, 1 for good reviews
model.to(device)

tokens = {k: v.to(device) for k, v in ttt.items()} #move token tensors to device

BaseModelOutput(last_hidden_state=tensor([[[-0.1970, -0.0701,  0.2239,  ...,  0.0774,  0.4518,  0.5697],
         [-0.4126, -0.7087,  0.9003,  ...,  0.2943,  0.5049,  0.7264],
         [-0.7741, -0.2630,  0.8735,  ...,  0.3536,  0.3339,  0.7135],
         ...,
         [-0.6673, -0.6769,  0.5810,  ...,  0.4725,  0.2744, -0.1187],
         [-0.3203, -0.1139,  0.2511,  ...,  0.3297,  0.4263,  0.3331],
         [ 0.7407,  0.2700,  0.0208,  ...,  0.2933, -0.2338, -0.1630]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


#### Exercise 1.3: A Stable Baseline

In this exercise I want you to:
1. Use Distilbert as a *feature extractor* to extract representations of the text strings from the dataset splits;
2. Train a classifier (your choice, by an SVM from Scikit-learn is an easy choice).
3. Evaluate performance on the validation and test splits.

These results are our *stable baseline* -- the **starting** point on which we will (hopefully) improve in the next exercise.

**Hint**: There are a number of ways to implement the feature extractor, but probably the best is to use a [feature extraction `pipeline`](https://huggingface.co/tasks/feature-extraction). You will need to interpret the output of the pipeline and extract only the `[CLS]` token from the *last* transformer layer. *How can you figure out which output that is?*

In [ ]:
# Feature extraction function

def compute_features(model, tokenizer, data_loader):
    model.eval()

    features = []

    with torch.no_grad(): 
        for batch in tqdm(data_loader):
            tokens = tokenizer(
                batch["text"],
                padding = True,
                truncation = True,
                return_tensors = "pt"
            )
            tokens = {k: v.to(device) for k, v in tokens.items()}

            outputs = model(**tokens)

            #extract the [CLS] token embedding (first token) from the last hidden state
            #we take all items in batch (:) , the first token (0), and all hidden dimensions (:)
            cls_embeds = outputs.last_hidden_state[:, 0, :].detach().cpu()

            features.append(cls_embeds)

            del tokens, outputs, cls_embeds 
            torch.cuda.empty_cache()

    features = torch.cat(features, dim = 0)

    return features

In [ ]:
# Train and evaluate small SVM classifier on the extracted features
clf = svm.SVC()

features = compute_features(model, tokenizer, train)

X = features.numpy()
y = np.array(dataset["train"]["label"])

clf.fit(X, y)

print("acc:", clf.score(X, y))

100%|██████████| 134/134 [00:02<00:00, 45.62it/s]


acc: 0.816295427901524


In [ ]:
#Test the classifier

features = compute_features(model, tokenizer, val)

X = features.numpy()
y = np.array(dataset["validation"]["label"])

print("acc:", clf.score(X, y))

y_pred  = clf.predict(X)
mis_idxs = np.where(y_pred != y)[0]
for i in mis_idxs[:n_examples_test]:
    i = int(i)
    print("Text :", dataset["validation"][i]["text"])
    print(f"Predicted  : { y_pred[i] }")

100%|██████████| 17/17 [00:00<00:00, 44.39it/s]


acc: 0.8142589118198874
Text : made for teens and reviewed as such , this is recommended only for those under 20 years of age . . . and then only as a very mild rental .
Predicted  : 0
Text : imagine o . henry's <b>the gift of the magi</b> relocated to the scuzzy underbelly of nyc's drug scene . merry friggin' christmas !
Predicted  : 0
Text : nothing short of wonderful with its ten-year-old female protagonist and its steadfast refusal to set up a dualistic battle between good and evil .
Predicted  : 0
Text : those moviegoers who would automatically bypass a hip-hop documentary should give " scratch " a second look .
Predicted  : 0
Text : baby-faced renner is eerily convincing as this bland blank of a man with unimaginable demons within .
Predicted  : 0


-----
### Exercise 2: Fine-tuning Distilbert

In this exercise we will fine-tune the Distilbert model to (hopefully) improve sentiment analysis performance.

#### Exercise 2.1: Token Preprocessing

The first thing we need to do is *tokenize* our dataset splits. Our current datasets return a dictionary with *strings*, but we want *input token ids* (i.e. the output of the tokenizer). This is easy enough to do my hand, but the HugginFace `Dataset` class provides convenient, efficient, and *lazy* methods. See the documentation for [`Dataset.map`](https://huggingface.co/docs/datasets/v3.5.0/en/package_reference/main_classes#datasets.Dataset.map).

**Tip**: Verify that your new datasets are returning for every element: `text`, `label`, `intput_ids`, and `attention_mask`.

In [ ]:
# Formatting dataset for tokenization

ds = dataset.map(lambda example: tokenizer(example["text"]), batched = True)
print(ds["train"][0])

Map: 100%|██████████| 1066/1066 [00:00<00:00, 53129.68 examples/s]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1, 'input_ids': [101, 1996, 2600, 2003, 16036, 2000, 2022, 1996, 7398, 2301, 1005, 1055, 2047, 1000, 16608, 1000, 1998, 2008, 2002, 1005, 1055, 2183, 2000, 2191, 1037, 17624, 2130, 3618, 2084, 7779, 29058, 8625, 13327, 1010, 3744, 1011, 18856, 19513, 3158, 5477, 4168, 2030, 7112, 16562, 2140, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


#### Exercise 2.2: Setting up the Model to be Fine-tuned

In this exercise we need to prepare the base Distilbert model for fine-tuning for a *sequence classification task*. This means, at the very least, appending a new, randomly-initialized classification head connected to the `[CLS]` token of the last transformer layer. Luckily, HuggingFace already provides an `AutoModel` for just this type of instantiation: [`AutoModelForSequenceClassification`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification). You will want you instantiate one of these for fine-tuning.

In [ ]:
# Load the pre-trained DistilBERT model for sequence classification

model_ds = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased") #(n labels is 2 by default)
model_ds.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


#### Exercise 2.3: Fine-tuning Distilbert

Finally. In this exercise you should use a HuggingFace [`Trainer`](https://huggingface.co/docs/transformers/main/en/trainer) to fine-tune your model on the Rotten Tomatoes training split. Setting up the trainer will involve (at least):


1. Instantiating a [`DataCollatorWithPadding`](https://huggingface.co/docs/transformers/en/main_classes/data_collator) object which is what *actually* does your batch construction (by padding all sequences to the same length).
2. Writing an *evaluation function* that will measure the classification accuracy. This function takes a single argument which is a tuple containing `(logits, labels)` which you should use to compute classification accuracy (and maybe other metrics like F1 score, precision, recall) and return a `dict` with these metrics.  
3. Instantiating a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.51.1/en/main_classes/trainer#transformers.TrainingArguments) object using some reasonable defaults.
4. Instantiating a `Trainer` object using your train and validation splits, you data collator, and function to compute performance metrics.
5. Calling `trainer.train()`, waiting, waiting some more, and then calling `trainer.evaluate()` to see how it did.

**Tip**: When prototyping this laboratory I discovered the HuggingFace [Evaluate library](https://huggingface.co/docs/evaluate/en/index) which provides evaluation metrics. However I found it to have insufferable layers of abstraction and getting actual metrics computed. I suggest just using the Scikit-learn metrics...

In [ ]:
# Metrics and Training setup

dcwp = DataCollatorWithPadding(tokenizer = tokenizer)

def compute_metrics(eval_pred, metrics_dict):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)

    metrics_results = {}
    for metric in metrics_dict.keys():
        metrics_results[metric] = metrics_dict[metric](preds, labels)

    return metrics_results

metrics = {
    "acc": lambda preds, labels: (preds == labels).mean(),
    "f1": lambda preds, labels: f1_score(labels, preds, average = "weighted"),
    "prec": lambda preds, labels: precision_score(labels, preds, average = "weighted"),
    "rec": lambda preds, labels: recall_score(labels, preds, average = "weighted")
}


training_args = TrainingArguments(
    output_dir = checkpoints_dir + "/fine_tuned_model",
    learning_rate = 2e-5,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 16,
    num_train_epochs = 5,
    weight_decay = 0.01,
    eval_strategy = "epoch", #evaluate at the end of each epoch
    save_strategy = "epoch",
    load_best_model_at_end = True,
    logging_dir = log_dir + "/fine_tuned_model",
    logging_strategy = "steps",
    logging_steps = 10,
    report_to = "tensorboard"
)

trainer = Trainer(
    model = model_ds,
    args = training_args,
    train_dataset = ds["train"],
    eval_dataset = ds["validation"],
    data_collator = dcwp,
    compute_metrics = lambda x: compute_metrics(x, metrics)
)

trainer.train()

Epoch,Training Loss,Validation Loss,Acc,F1,Prec,Rec
1,0.425300,0.372811,0.836773,0.836106,0.842345,0.836773
2,0.237100,0.349650,0.848968,0.848920,0.849412,0.848968
3,0.169500,0.452462,0.840525,0.840457,0.841106,0.840525
4,0.047400,0.499667,0.852720,0.852720,0.852722,0.852720
5,0.066700,0.545384,0.848968,0.848957,0.849068,0.848968


TrainOutput(global_step=1335, training_loss=0.2043413557251741, metrics={'train_runtime': 57.4467, 'train_samples_per_second': 742.427, 'train_steps_per_second': 23.239, 'total_flos': 580344848019696.0, 'train_loss': 0.2043413557251741, 'epoch': 5.0})

-----
### Exercise 3: Choose at Least One


#### Exercise 3.1: Efficient Fine-tuning for Sentiment Analysis (easy)

In Exercise 2 we fine-tuned the *entire* Distilbert model on Rotten Tomatoes. This is expensive, even for a small model. Find an *efficient* way to fine-tune Distilbert on the Rotten Tomatoes dataset (or some other dataset).

**Hint**: You could check out the [HuggingFace PEFT library](https://huggingface.co/docs/peft/en/index) for some state-of-the-art approaches that should "just work". How else might you go about making fine-tuning more efficient without having to change your training pipeline from above?

In [ ]:
# LoRa fine-tuning

# Redefine the model for LoRa fine-tuning
model_ds = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 2)
model_ds.to(device)

lora_config = LoraConfig(    
    r = 8, #LoRa rank decompostion
    lora_alpha = 16, #Suggested value (no clue why)
    target_modules = ["q_lin", "v_lin"],
    lora_dropout = 0.1,
    bias = "none",
    task_type = TaskType.SEQ_CLS
)

model_ds = get_peft_model(model_ds, lora_config)
model_ds.print_trainable_parameters()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


In [ ]:
# Train the LoRa model

training_args = TrainingArguments(
    output_dir = checkpoints_dir + "/lora_model",
    learning_rate = 2e-5,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 16,
    num_train_epochs = 3,
    weight_decay = 0.01,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    logging_dir = log_dir + "/lora_model",
    logging_strategy = "steps",
    logging_steps = 10,
    report_to = "tensorboard"
)

trainer = Trainer(
    model = model_ds,
    args = training_args,
    train_dataset = ds["train"],
    eval_dataset = ds["validation"],
    data_collator = dcwp,
    compute_metrics = lambda x: compute_metrics(x, metrics)
)

trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Acc,F1,Prec,Rec
1,0.508400,0.468388,0.797373,0.797373,0.797378,0.797373
2,0.450500,0.424912,0.813321,0.813071,0.815007,0.813321
3,0.442200,0.420202,0.814259,0.814023,0.815864,0.814259


TrainOutput(global_step=801, training_loss=0.49919321578688985, metrics={'train_runtime': 20.3595, 'train_samples_per_second': 1256.906, 'train_steps_per_second': 39.343, 'total_flos': 354239374467936.0, 'train_loss': 0.49919321578688985, 'epoch': 3.0})

#### Exercise 3.2: Fine-tuning a CLIP Model (harder)

Use a (small) CLIP model like [`openai/clip-vit-base-patch16`](https://huggingface.co/openai/clip-vit-base-patch16) and evaluate its zero-shot performance on a small image classification dataset like ImageNette or TinyImageNet. Fine-tune (using a parameter-efficient method!) the CLIP model to see how much improvement you can squeeze out of it.

**Note**: There are several ways to adapt the CLIP model; you could fine-tune the image encoder, the text encoder, or both. Or, you could experiment with prompt learning.

**Tip**: CLIP probably already works very well on ImageNet and ImageNet-like images. For extra fun, look for an image classification dataset with different image types (e.g. *sketches*).

In [ ]:
# Your code here.

#### Exercise 3.3: Choose your Own Adventure

There are a *ton* of interesting and fun models on the HuggingFace hub. Pick one that does something interesting and adapt it in some way to a new task. Or, combine two or more models into something more interesting or fun. The sky's the limit.

**Note**: Reach out to me by email or on the Discord if you are unsure about anything.

In [ ]:
# Your code here.